In [46]:
import pandas as pd
import numpy as np

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [23]:
df = pd.read_csv('data/output_w_label.csv')

In [24]:
df = df.drop(columns='Unnamed: 0')

In [25]:
df["transcript_no"] = df["ID"].str.extract(r"^([A-Z]+)(\d+)")[1]

df["gene_no"] = df["gene_id"].str.extract(r"^([A-Z]+)(\d+)")[1]

In [26]:
# NOTE: In the dataset currently all follow DRACH for middle 5-mer

# import itertools

# # Define possibilities
# D = ['A', 'G', 'T']   # not C
# R = ['A', 'G']
# A = ['A']
# C = ['C']
# H = ['A', 'C', 'T']   # not G

# # Generate all combinations
# drach_motifs = [''.join(p) for p in itertools.product(D, R, A, C, H)]
# print(len(drach_motifs), drach_motifs)  # should be 18

In [27]:
# def middle_is_drach(seq):
#     return seq[1:len(seq)-1] in drach_motifs

In [28]:
# df["middle_is_drach"] = df["SEQ"].apply(middle_is_drach)

In [29]:
df_final = df.drop(columns=['ID', 'gene_id'], axis=1)

In [30]:
df_final.head()

,POS,SEQ,PreTime,PreSD,PreMean,InTime,InSD,InMean,PostTime,PostSD,PostMean,label,transcript_no,gene_no
0,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,0,00000000233,00000004059
1,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,0,00000000233,00000004059
2,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,0,00000000233,00000004059
3,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,0,00000000233,00000004059
4,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,0,00000000233,00000004059


In [31]:
# df_final[df_final['middle_is_drach']==False]

In [32]:
# len(df_final[df_final['middle_is_drach']==False])

In [33]:
X = df_final.drop(columns=['label', 'SEQ'], axis=1)
y = df_final['label']

In [34]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)
X_train, y_train = X_train_val, y_train_val

In [35]:
# svm_model = svm.SVC()

In [36]:
# svm_model.fit(X_train, y_train)

In [37]:
# y_pred_test = svm_model.predict(X_test)

# auc_score = roc_auc_score(y_test, y_pred_test)
# print(f"ROC AUC Score: {auc_score}")

In [48]:
rf = RandomForestClassifier(max_depth=5, random_state=42)

In [49]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [50]:
# ROC AUC
roc_auc_scores = cross_val_score(rf, X, y, cv=skf, scoring="roc_auc")

# Precision-Recall AUC
pr_auc_scores = cross_val_score(rf, X, y, cv=skf, scoring="average_precision")

print("ROC AUC per fold:", roc_auc_scores)
print("Mean ROC AUC:", np.mean(roc_auc_scores))

print("PR AUC per fold:", pr_auc_scores)
print("Mean PR AUC:", np.mean(pr_auc_scores))

KeyboardInterrupt: 

In [39]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=42)

In [40]:
y_pred_test = rf.predict(X_test)

auc_score = roc_auc_score(y_test, y_pred_test)
print(f"ROC AUC Score: {auc_score}")

ROC AUC Score: 0.5


In [44]:
pr_score = average_precision_score(y_test, y_pred_test)
print(f"PR AUC Score: {pr_score}")

PR AUC Score: 0.04530425469592667
